## Python 2 用的是深度优先，从左到右
## Python 3 之后用的是C3 Linearization Algorithm
## 那么问题在于，什么是一个好的节点？我们说 N 是一个好的节点当且仅当搜索路径中 N 之后的节点都不继承自 N。我们还以上述的类继承图为例，按照深度优先遍历得到类 D 中函数的搜索路径 D, B, A, C, A。之后 Python 解释器从左向右检查是一个好的节点，因为 A 之后的节点 C 继承自 A。因此其将 A 从搜索路径中移除，然后得到最后的调用顺序 D, B, C, A。

In [1]:
class A:
    def process(self):
        print('A process()')


class B:
    pass


class C(A, B):
    pass


obj = C()  
obj.process()    
print(C.mro())

A process()
[<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class 'object'>]


In [2]:
C.mro

<function C.mro()>

In [5]:
class A:
    def process(self):
        print('A process()')


class B(A):
    def process(self):
        print('B process()')


class C(A, B):
    pass


obj = C()
obj.process()

TypeError: Cannot create a consistent method resolution
order (MRO) for bases A, B

## The problem comes from the fact that class A is a super class for both C and B. If you construct MRO then it should be like this:
## C -> A -> B -> A

## Then according to the rule (good head) A should NOT be ahead of B as A is super class of B. So new MRO must be like this:
## C -> B -> A 

## But A is also direct super class of C. So, if a method is in both A and B classes then which version should class C call? According to new MRO, the version in B is called first ahead of A and that is not according to inheritance rules (specific to generic) resulting in Python to throw error.

## 图在最下面： http://www.srikanthtechnologies.com/blog/python/mro.aspx

In [7]:
# 这个例子的讲解 http://kaiyuan.me/2016/04/27/C3_linearization/

class X():
    def who_am_i(self):
        print("I am a X")
        
class Y():
    def who_am_i(self):
        print("I am a Y")
        
class A(X, Y):
    def who_am_i(self):
        print("I am a A")
        
class B(Y, X):
     def who_am_i(self):
         print("I am a B")
         
class F(A, B):
    def who_am_i(self):
        print("I am a F")

TypeError: Cannot create a consistent method resolution
order (MRO) for bases X, Y